In [2]:
from easygui import *
import pyxdf
import PyQt5
import mne
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import pathlib
import mne
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#from joblib import dump,load
import pickle
# For interactive plots
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'qt')
#from subfunctions import *


from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, LinearModel, get_coef,
                          Vectorizer, CSP)

import warnings
warnings.filterwarnings('ignore')
import os,numpy as np,pandas as pd
from collections import OrderedDict
import seaborn as sns
from matplotlib import pyplot as plt

# MNE functions
from mne import Epochs,find_events
from mne.decoding import Vectorizer


# Scikit-learn and Pyriemann ML functionalities
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from pyriemann.estimation import ERPCovariances, XdawnCovariances, Xdawn
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from sklearn.metrics import confusion_matrix



In [3]:
class BandwidthFilter:
    def __init__(self, l_freq, h_freq, method):
        self.l_freq = l_freq
        self.h_freq = h_freq
        self.method = method
        
class model:
    def __init__(self, clsf, input_shape, ch_names,bandwidthfilter):
        self.clsf = clsf
        self.input_shape = input_shape
        self.ch_names = ch_names
        self.bandwidthfilter = bandwidthfilter



In [4]:
processed_file_name = 'Omri_Recording_001'

In [5]:
current_path = pathlib.Path().absolute()  
data_fname = current_path /'Data'/'Processed Data'/ (processed_file_name + '_Processed')
epochs = mne.read_epochs(data_fname)

#read the pickle file
picklefile = open(current_path /'Data'/'Processed Data'/ (processed_file_name+'_Filter'), 'rb')
#unpickle the dataframe
bandwidth_filter = pickle.load(picklefile)
#close file
picklefile.close()

#bandwidth_filter = load(current_path /'Data'/'Processed Data'/ (processed_file_name+'_Filter'))

Reading c:\Users\user\Documents\BCI_ALS\BCI_ALS\Data\Processed Data\Omri_Recording_001_Processed ...
    Found the data of interest:
        t =    -200.00 ...     496.00 ms
        0 CTF compensation matrices available
Not setting metadata
277 matching events found
No baseline correction applied
0 projection items activated


In [6]:
X = epochs.get_data()  # MEG signals: n_epochs, n_meg_channels, n_times
# Remove baslineperiod
baseline_duration = -0.2
onset_sample = int(np.absolute(baseline_duration) * 125)
X = X[:,:,onset_sample::]

y = epochs.events[:, 2]  # target: auditory left vs visual left
y = y-1 # 0's and 1's instead of 1's and 2's


In [8]:
clfs = OrderedDict()
clfs['Vect + LR'] = make_pipeline(Vectorizer(), StandardScaler(), LogisticRegression())
clfs['Vect + RegLDA'] = make_pipeline(Vectorizer(), LDA(shrinkage='auto', solver='eigen'))
clfs['Xdawn + RegLDA'] = make_pipeline(Xdawn(2, classes=[1]), Vectorizer(), LDA(shrinkage='auto', solver='eigen'))

clfs['XdawnCov + TS'] = make_pipeline(XdawnCovariances(estimator='oas'), TangentSpace(), LogisticRegression())
clfs['XdawnCov + MDM'] = make_pipeline(XdawnCovariances(estimator='oas'), MDM())


clfs['ERPCov + TS'] = make_pipeline(ERPCovariances(), TangentSpace(), LogisticRegression())
clfs['ERPCov + MDM'] = make_pipeline(ERPCovariances(), MDM())


# format data
epochs.pick_types(eeg=True)
X = epochs.get_data() * 1e6
baseline_duration = -0.2
onset_sample = int(np.absolute(baseline_duration) * 125)
X = X[:,:,onset_sample::]

times = epochs.times
y = epochs.events[:, -1]
y = y-1

# define cross validation
cv = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

# run cross validation for each pipeline
auc = []
methods = []
for m in clfs:
    res = cross_val_score(clfs[m], X, y==1, scoring='roc_auc', cv=cv, n_jobs=-1)
    auc.extend(res)
    methods.extend([m]*len(res))

results = pd.DataFrame(data=auc, columns=['AUC'])
results['Method'] = methods

plt.figure(figsize=[8,4])
sns.barplot(data=results, x='AUC', y='Method')
plt.xlim(0.2, 0.85)
sns.despine()


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\BCI-ALS\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\user\anaconda3\envs\BCI-ALS\lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\user\anaconda3\envs\BCI-ALS\lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\user\anaconda3\envs\BCI-ALS\lib\site-packages\joblib\memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "c:\Users\us

In [13]:
len(y)

277

## Lets fit the model to the data and save it

### First, Choose the best model to save

In [50]:
pipe = clfs['Vect + LR'] 

In [51]:
pipe.fit(X,y)
input_shape = X.shape
ch_names = epochs.ch_names


m = model(pipe,input_shape,ch_names,bandwidth_filter)


fname = processed_file_name+'_model'
path_fname = current_path /'Models'/ fname


#create a pickle file
picklefile = open(path_fname, 'wb')
#pickle the dictionary and write it to file
pickle.dump(m, picklefile)
#close the file
picklefile.close()

In [52]:
#read the pickle file
picklefile = open(path_fname, 'rb')
#unpickle the dataframe
loaded = pickle.load(picklefile)
#close file
picklefile.close()